# W2 - Geometry & Topoology

In this workshop we will learn about mesh data structures, how to import, visualize and export them in python. We will also learn about basic mesh manipulations.

## 0. Initialization

### 0.1. Importing libraries 

In [37]:
# basic libraries
import numpy as np

# computational geometry libraries
import pyvista as pv
from compas.datastructures import Mesh as CompasMesh
from compas_plotters.meshplotter import MeshPlotter

# libraries for connection to grasshopper (speckle)
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.transports.server import ServerTransport
from specklepy.api import operations

# Earthy Bridge
import earthy_bridge as eb

# extra IPython utilities
from IPython.display import display

### 0.2. Establish connection to speckle

In [38]:
# initialise the client
client = SpeckleClient(host="speckle.xyz", use_ssl=True)

# authenticate the client with a token
account = get_default_account()
client.authenticate(token=account.token)

# use that stream id to get the stream from the server
earthy_w2_stream = client.stream.get(id="5be382cdb7")

# creating a server transport 
transport = ServerTransport(client=client, stream_id=earthy_w2_stream.id)

## 1. Mesh In

### 1.1. Importing OBJ files

In [39]:
# loading the mesh
base_grid_filepath = "../data/base_grid.obj"
base_grid_mesh = CompasMesh.from_obj(base_grid_filepath)
# visualizing the mesh
eb.compas_to_pyvista(base_grid_mesh).plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### 1.2. Importing from Grasshopper (Speckle)

In [40]:
# find base box reference id
last_geo_ref = client.commit.list(earthy_w2_stream.id)[0].referencedObject

# this receives the object back from the transport.
received_mesh = operations.receive(obj_id=last_geo_ref, remote_transport=transport)

In [41]:
# visualize the mesh
eb.speckle_to_pyvista(received_mesh).plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 2. Mesh Manipulation & Analysis

### 2.1. Mesh Data Structure 

In [42]:
# construct the compas mesh
(V, F) = eb.speckle_to_vertices_and_faces(received_mesh)

# display("Vertices: ", V)
# display("Faces: ", F)

### 2.2. Basic Mesh Transformation

In [43]:
# scale in Y direction
edited_V = np.array(V) * np.array([1,1.5,1])

In [44]:
pv_edited_mesh = eb.vertices_and_faces_to_pyvista(edited_V, F)
pv_edited_mesh.plot(color='#abd8ff', show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### 2.3. Dual

In [58]:
# construct the compas mesh
edited_mesh_compas = CompasMesh.from_vertices_and_faces(edited_V,F)
received_mesh_compas = CompasMesh.from_vertices_and_faces(V,F)

In [46]:
dual_mesh = received_mesh_compas.dual()
dV, dF = dual_mesh.to_vertices_and_faces()

In [56]:
p = pv.Plotter()
p.add_mesh(eb.compas_to_pyvista(received_mesh_compas), opacity=0.5, show_edges=True)
p.add_mesh(eb.compas_to_pyvista(dual_mesh), style="wireframe")
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
# (Q+) can you propose an algorithm/procedure to generate the dual of a mesh?

## 3. Mesh Out

### 3.1. Saving OBJ files

In [59]:
# save the mesh into an OBJ file
edited_mesh_compas.to_obj("../data/edited_mesh.obj")

### 3.2. Exporting to Grasshopper (Speckle)

In [61]:
# construct speckle mesh
edited_mesh_speckle = eb.compas_to_speckle(edited_mesh_compas)

In [62]:
# Serialises the block and sends it to the transport
edited_mesh_hash = operations.send(base=edited_mesh_speckle, transports=[transport])

# creating a commit on your stream with this object
commid_id = client.commit.create(
    stream_id=earthy_w2_stream.id, 
    object_id=edited_mesh_hash, 
    message="edited_mesh")

## Credits

In [ ]:
__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/earthy_workshops"
__summary__ = "Earthy Design Studio Workshop on Geometry and Topology"